## Importación de librerías 

En esta sección se importan las principales librerías utilizadas en el preprocesamiento del texto.
- **spaCy**: biblioteca de procesamiento del lenguaje natural (NLP) que permite realizar tareas como tokenización, eliminación de stopwords o lematización.
    - Se incluye además la configuración personalizada del tokenizador de spaCy, que permite ajustar cómo se separan los tokens (por ejemplo, evitar divisiones innecesarias en palabras con guiones, puntos, etc.).
- **pandas**: biblioteca para la manipulación de datos estructurados (tablas) de tipo DataFrame.
- **collections.Counter**: conteo eficiente de elementos en colecciones.
- **sklearn.preprocessing.normalize**: normalización de vectores o matrices.
- **sklearn.metrics.pairwise.cosine_similarity**: cálculo de similitud coseno entre vectores.


In [6]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
# Librería que contiene herramientas de preprocesamiento
import spacy

# Herramientas internas de spaCy para modificar el tokenizador por defecto
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

# Librería para carga de datos
import pandas as pd

# Librería para contar frecuencias
from collections import Counter

# Librerías para normalizar y aplicar similitud del coseno
import sklearn
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

spaCy: 3.8.4
pandas: 2.2.3
scikit-learn: 1.6.1


## Definición de funciones y configuración de preprocesamiento

Este bloque incluye todo lo necesario para preparar el texto antes de su análisis. Se utiliza el modelo `en_core_web_lg` de spaCy y se personaliza el tokenizador para evitar divisiones en guiones.

### Componentes definidos:

- **Carga y personalización del modelo spaCy**: se modifica el tokenizador para no dividir palabras por guiones.
- **`tokenizar_texto_spacy(texto)`**: convierte una cadena de texto a minúsculas y la transforma en una lista de tokens.
- **`eliminar_stopwords_spacy(tokens)`**: elimina tokens irrelevantes, incluyendo stopwords, palabras no alfabéticas y una lista personalizada de palabras excluidas.
- **`lematizar_texto_spacy(tokens)`**: transforma cada token en su lema o raíz.
- **`aplicar_mapeo_df(serie_de_listas, mapeo)`**: aplica un diccionario de equivalencias léxicas para unificar términos similares.
- **`palabras_extra`**: lista manual de palabras irrelevantes específicas del contexto del corpus.
- **`mapeo`**: diccionario de normalización construido a partir de un archivo `.csv` externo.


In [2]:
# Cargar el modelo grande de inglés (incluye vectores de palabras)
nlp = spacy.load("en_core_web_lg")

# Eliminar los guiones como separadores internos para evitar fragmentación de tokens como "ai-based" en "ai" "based"
infixes = [x for x in nlp.Defaults.infixes if "-" not in x]
infix_re = compile_infix_regex(infixes)

# Reconfigurar el tokenizador con las nuevas reglas de infijos
nlp.tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=nlp.tokenizer.prefix_search,
    suffix_search=nlp.tokenizer.suffix_search,
    infix_finditer=infix_re.finditer,
    token_match=nlp.tokenizer.token_match
)



# Función de tokenización básica con spaCy en la que también se pasa el texto en minúsculas
# La separación en tokens se realiza según las reglas del modelo de lenguaje cargado,
# teniendo en cuenta espacios, puntuación, signos de puntuación interna (como guiones, apóstrofes, etc.)
# y excepciones predefinidas. En este caso, se ha modificado el tokenizer para que no divida por guiones.
def tokenizar_texto_spacy(texto):
    doc = nlp(texto.lower())
    return list(doc)




# Lista personalizada de palabras a ignorar (además de las stopwords de spaCy)
palabras_extra = [
    "co", "digital", "alexa", "anytime", "cortana", "arduino", "bitcoin",
    "deepl", "digitally", "digitise", "dropbox", "duckduckgo", "e", "etc",
    "eurostat", "example", "facebook", "google", "miro", "t", "twitter",
    "tpm", "youtube", "vs", "whatsapp", "wikipedia", "x", "en", "se", "lego",
    "non", "python", "ros", "scratch", "siri", "-", "non-digital", "one-time",
    "bas", "digital-based", "digitise", "digitised", "digitization", "s", "whilst"
]

# Asegurar que "3d" y "3-d" no se consideren stopwords, ya que el eliminador de stopwrods 
# elimina todas con números
nlp.vocab["3d"].is_alpha = True
nlp.vocab["3-d"].is_alpha = True
nlp.vocab["2d"].is_alpha = True
nlp.vocab["2-d"].is_alpha = True

# Función para eliminar stopwords y tokens no alfabéticos.
# Ignora las palabras_extra
def eliminar_stopwords_spacy(tokens):
    return [
        token for token in tokens
        if not token.is_stop
        and token.is_alpha
        and token.text.lower() not in palabras_extra
    ]
    
    
    
    
# Función de lematización del modelos cargado
def lematizar_texto_spacy(tokens):
    return [token.lemma_ for token in tokens]


#carga del archivo que contiene los mapeos correspondientes y conversión a tipo DataFrame para ser tratado como tabla
ruta_mapeo = "../utils/mapping.csv"
df_mapeo = pd.read_csv(ruta_mapeo)

# Construcción del diccionario: original -> equivalente
mapeo = dict(zip(df_mapeo["original"], df_mapeo["equivalente"]))

## Función `comparar_texto_con_frecuencias`

Compara un texto con grupos de una tabla de frecuencias y calcula la similitud basada en palabras comunes.

**Parámetros:**

- `texto`: lista de palabras del texto preprocesado.
- `df_pivot`: DataFrame con palabras como filas y grupos como columnas, con frecuencias.
- `top_n`: número de palabras clave a mostrar por grupo (por defecto 5).

**Funcionamiento resumido:**

1. Obtiene el vocabulario del DataFrame.
2. Crea un vector con las frecuencias de las palabras del texto en el vocabulario.
3. Combina este vector con los vectores de frecuencias de los grupos.
4. Normaliza los vectores para calcular similitud coseno.
5. Calcula la similitud entre el texto y cada grupo.
6. Identifica las palabras que más contribuyen a la similitud para cada grupo.
7. Devuelve un DataFrame con grupo, similitud y palabras clave ordenado por similitud.

**Devuelve:**

Un DataFrame con columnas: `'Grupo'`, `'Similitud'` y `'Palabras_clave'`.

## Función `aplicar_mapeo_a_texto(lista_tokens, mapeo)`

Reemplaza cada token en una lista de estos según un diccionario de mapeo léxico para unificar términos similares.


In [3]:
def comparar_texto_con_frecuencias(texto, df_pivot, top_n=5):

    # 1. Vocabulario (palabras que aparecen en df_pivot)
    vocabulario = df_pivot.index.tolist()

    # 2. Vector del texto: contar solo palabras del vocabulario
    conteo_texto = Counter(p for p in texto if p in vocabulario)
    vector_texto = pd.Series([conteo_texto.get(p, 0) for p in vocabulario], index=vocabulario).to_frame(name="Texto")

    # 3. Concatenar vectores del texto y de grupos
    matriz_completa = pd.concat([df_pivot, vector_texto], axis=1).fillna(0)

    # 4. Normalizar vectores para cálculo coseno
    matriz_normalizada = normalize(matriz_completa.T)

    # 5. Calcular similitud coseno entre texto (última fila) y cada grupo (todas menos última)
    similitudes = cosine_similarity([matriz_normalizada[-1]], matriz_normalizada[:-1])[0]

    # 6. Preparar resultados básicos
    nombres_grupo = df_pivot.columns.tolist()
    resultados = pd.DataFrame({
        'Grupo': [g if isinstance(g, str) else ' - '.join(map(str, g)) for g in nombres_grupo],
        'Similitud': similitudes
    })

    # 7. Calcular contribuciones por palabra a la similitud de cada grupo
    #    La contribución para cada palabra = frecuencia_texto * frecuencia_grupo
    #    Ordenamos y extraemos las top_n palabras para cada grupo
    palabras_clave_por_grupo = []
    for grupo in nombres_grupo:
        contribuciones = []
        for palabra in vocabulario:
            f_texto = vector_texto.at[palabra, "Texto"]
            f_grupo = df_pivot.at[palabra, grupo]
            if f_texto > 0 and f_grupo > 0:
                contribuciones.append((palabra, f_texto * f_grupo))

        # Ordenar por contribución descendente
        contribuciones.sort(key=lambda x: x[1], reverse=True)
        top_palabras = [p for p, _ in contribuciones[:top_n]]
        palabras_clave_por_grupo.append(", ".join(top_palabras))

    resultados["Palabras_clave"] = palabras_clave_por_grupo

    # 8. Ordenar resultados por similitud descendente
    resultados = resultados.sort_values(by="Similitud", ascending=False).reset_index(drop=True)

    return resultados

def aplicar_mapeo_a_texto(lista_tokens, mapeo):
    return [mapeo.get(token, token) for token in lista_tokens]

## Bloque para modificar el texto a analizar

En este bloque el usuario debe introducir el texto que desea analizar. El texto pasará por varias etapas de preprocesamiento: tokenización, eliminación de stopwords, lematización y aplicación del mapeo léxico definido.


In [4]:
# TEXTO A COMPARAR
texto = "In today digital society, individuals need to use various tools and applications to access, manage, and evaluate information efficiently. Being able to find trustworthy sources, use appropriate technologies, and apply knowledge critically is essential. People should also understand how digital tools influence behavior, communication, and decision-making. The ability to use AI systems and understand data privacy is increasingly important in both education and work contexts."
texto1 = tokenizar_texto_spacy(texto)
texto2 = eliminar_stopwords_spacy(texto1)
texto3 = lematizar_texto_spacy(texto2)
texto4 = aplicar_mapeo_a_texto(texto3, mapeo)

## Comparación de texto seleccionado con texto de DigComp

### Dimensión 2 agrupado por áreas

In [5]:
ruta_d2_areas = "../results/digcomp/frecuencias_d2_areas.csv"
frecuencias_2_areas = pd.read_csv(ruta_d2_areas, index_col=0)

resultados_d2_areas = comparar_texto_con_frecuencias(texto4, frecuencias_2_areas, top_n=5)
print(resultados_d2_areas)

  Grupo  Similitud                                   Palabras_clave
0     2   0.543914         use, knowledge, tool, communicate, datum
1     5   0.434283     use, tool, knowledge, technology, understand
2     4   0.357486  use, datum, knowledge, appropriate, application
3     1   0.327865       datum, use, information, knowledge, access
4     3   0.250024              use, knowledge, datum, access, find


### Dimensión 2 agrupado por competencias

In [6]:
ruta_d2_competencias = "../results/digcomp/frecuencias_d2_competencia.csv"
frecuencias_2_competencias = pd.read_csv(ruta_d2_competencias, index_col=0)

resultados_d2_competencias = comparar_texto_con_frecuencias(texto4, frecuencias_2_competencias, top_n=5)
print(resultados_d2_competencias)

   Grupo  Similitud                                     Palabras_clave
0    2.4   0.471332            use, tool, knowledge, work, appropriate
1    5.4   0.435710              use, tool, understand, ai, technology
2    5.2   0.374324                 use, tool, knowledge, need, access
3    5.3   0.350115      use, tool, knowledge, technology, information
4    2.1   0.349856     use, communicate, tool, knowledge, appropriate
5    2.3   0.298903                use, ai, knowledge, society, system
6    1.2   0.277407          information, source, datum, knowledge, ai
7    4.3   0.275510     use, ability, behaviour, knowledge, technology
8    1.3   0.258482                 datum, use, application, knowledge
9    2.6   0.247076              datum, use, knowledge, manage, system
10   4.4   0.225460                  use, knowledge, datum, technology
11   1.1   0.224129         use, information, access, datum, knowledge
12   2.5   0.213158  behaviour, communicate, people, knowledge, manage
13   3

### Dimensión 3 agrupado por competencias

In [7]:
ruta_d3_competencias = "../results/digcomp/frecuencias_d3_competencia.csv"
frecuencias_3_competencias = pd.read_csv(ruta_d3_competencias, index_col=0)

resultados_d3_competencias = comparar_texto_con_frecuencias(texto4, frecuencias_3_competencias, top_n=5)
print(resultados_d3_competencias)

   Grupo  Similitud                                 Palabras_clave
0    4.4   0.350339                                use, technology
1    2.4   0.310835                               tool, technology
2    1.2   0.299945         datum, information, evaluation, source
3    5.2   0.279389                         need, tool, technology
4    2.1   0.278423  technology, communicate, appropriate, context
5    5.3   0.253024                    tool, technology, knowledge
6    2.3   0.241227               society, technology, appropriate
7    3.3   0.220755                      apply, information, datum
8    2.2   0.212664           technology, information, appropriate
9    1.1   0.205181                             information, datum
10   4.2   0.167248                                 datum, privacy
11   2.6   0.159111                                    tool, datum
12   1.3   0.134249                             datum, information
13   4.3   0.092547                                     techno

### Dimensión 3 agrupado por nivel de competencia

In [8]:
ruta_d3_tipoNivel = "../results/digcomp/frecuencias_d3_tipoNivel.csv"
frecuencias_3_tipoNivel = pd.read_csv(ruta_d3_tipoNivel, index_col=0)

resultados_d3_tipoNivel = comparar_texto_con_frecuencias(texto4, frecuencias_3_tipoNivel, top_n=5)
print(resultados_d3_tipoNivel)

                Grupo  Similitud                              Palabras_clave
0            Advanced   0.496906  appropriate, use, datum, technology, apply
1        Intermediate   0.235273        information, technology, datum, need
2          Foundation   0.131585              technology, datum, information
3  Highly specialised   0.127935                       knowledge, technology


### Dimensión 3 agrupado por subnivel de competencias

In [9]:
ruta_d3_subnivel = "../results/digcomp/frecuencias_d3_subnivel.csv"
frecuencias_3_subnivel = pd.read_csv(ruta_d3_subnivel, index_col=0)

resultados_d3_subnivel = comparar_texto_con_frecuencias(texto4, frecuencias_3_subnivel, top_n=5)
print(resultados_d3_subnivel)

  Grupo  Similitud                               Palabras_clave
0     5   0.375293   apply, use, datum, information, technology
1     6   0.352262  appropriate, technology, datum, information
2     4   0.285143         information, technology, datum, need
3     7   0.169703                        knowledge, technology
4     3   0.141208               technology, information, datum
5     2   0.131585               technology, datum, information
6     1   0.131585               technology, datum, information
7     8   0.000000                                             


### Dimensión 4 agrupado por competencias

In [10]:
ruta_d4_competencias = "../results/digcomp/frecuencias_d4_competencias.csv"
frecuencias_4_competencias = pd.read_csv(ruta_d3_competencias, index_col=0)

resultados_d4_competencias = comparar_texto_con_frecuencias(texto4, frecuencias_4_competencias, top_n=5)
print(resultados_d4_competencias)

   Grupo  Similitud                                 Palabras_clave
0    4.4   0.350339                                use, technology
1    2.4   0.310835                               tool, technology
2    1.2   0.299945         datum, information, evaluation, source
3    5.2   0.279389                         need, tool, technology
4    2.1   0.278423  technology, communicate, appropriate, context
5    5.3   0.253024                    tool, technology, knowledge
6    2.3   0.241227               society, technology, appropriate
7    3.3   0.220755                      apply, information, datum
8    2.2   0.212664           technology, information, appropriate
9    1.1   0.205181                             information, datum
10   4.2   0.167248                                 datum, privacy
11   2.6   0.159111                                    tool, datum
12   1.3   0.134249                             datum, information
13   4.3   0.092547                                     techno

### Dimensión 4 agrupado por tipo de ejemplos

In [11]:
ruta_d4_tipoEjemplo = "../results/digcomp/frecuencias_d4_tipoEjemplo.csv"
frecuencias_4_tipoEjemplo = pd.read_csv(ruta_d4_tipoEjemplo, index_col=0)

resultados_d4_tipoEjemplo = comparar_texto_con_frecuencias(texto4, frecuencias_4_tipoEjemplo, top_n=5)
print(resultados_d4_tipoEjemplo)

       Grupo  Similitud                           Palabras_clave
0     Skills   0.533470     use, knowledge, tool, ability, datum
1  Knowledge   0.419379          use, knowledge, datum, tool, ai
2  Attitudes   0.411161  use, information, ai, technology, datum
